In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# ==========================================
# 1. 사용자 설정 (이 부분을 꼭 수정하세요!)
# ==========================================
# 학습된 모델 경로 (보통 runs/obb/train/weights/best.pt 에 저장됩니다)
MODEL_PATH = "runs/obb/train/weights/best.pt" 

# [숙제]에서 구한 값들을 여기에 적어주세요
KNOWN_WIDTH = 5.0      # 물체의 실제 크기 (단위: cm)
FOCAL_LENGTH = 600     # 1단계 공식으로 계산한 초점 거리 값

# ==========================================
# 2. 거리 계산 함수
# ==========================================
def calculate_distance(pixel_width):
    if pixel_width <= 0: return 0
    # 공식: 거리 = (실제크기 * 초점거리) / 화면상픽셀크기
    return (KNOWN_WIDTH * FOCAL_LENGTH) / pixel_width

# ==========================================
# 3. 메인 실행 코드
# ==========================================
# 모델 로드
print(">>> 모델을 로드 중입니다...")
model = YOLO(MODEL_PATH)

# 카메라 열기 (0번은 보통 기본 웹캠, JETANK의 경우 CSICamera 설정을 따를 수도 있음)
cap = cv2.VideoCapture(0)

# 해상도 설정 (속도를 위해 적절히 조절, 학습 시 imgsz=224와는 별개)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print(">>> 실행 중... (종료하려면 'q'를 누르세요)")

while True:
    ret, frame = cap.read()
    if not ret: break

    # YOLO 추론 (학습할 때 imgsz=224로 했으므로 여기서도 맞춰주는 게 정확도에 좋습니다)
    results = model(frame, imgsz=224, verbose=False)

    # 결과 처리
    for result in results:
        # OBB(회전 박스) 감지된 객체가 있을 경우
        if result.obb is not None:
            # 여러 개가 잡힐 수 있으므로 반복문
            for i, box in enumerate(result.obb):
                
                # 1. OBB 좌표 정보 가져오기 (x, y, w, h, rotation)
                # xywhr 포맷: 중심x, 중심y, 너비, 높이, 회전각
                w = float(box.xywhr[0][2])
                h = float(box.xywhr[0][3])
                
                # 2. 픽셀 크기 결정
                # 정육면체 박스라면 w와 h 중 더 길게 잡힌 쪽을 기준으로 삼는 게 안정적입니다.
                pixel_size = max(w, h)
                
                # 3. 거리 계산
                distance = calculate_distance(pixel_size)
                
                # 4. 화면에 그리기
                # OBB의 4개 꼭짓점 좌표를 가져와서 그립니다
                corners = box.xyxyxyxy[0].cpu().numpy().astype(int)
                cv2.polylines(frame, [corners], isClosed=True, color=(0, 255, 0), thickness=2)
                
                # 중심점 계산
                center_x = int(box.xywhr[0][0])
                center_y = int(box.xywhr[0][1])
                
                # 텍스트 표시 (거리)
                label = f"{distance:.1f} cm"
                cv2.putText(frame, label, (center_x, center_y - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                print(f"감지됨! 거리: {distance:.1f} cm, 픽셀크기: {pixel_size:.1f}")

    # 화면 출력
    cv2.imshow("OBB Distance Measurement", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()